In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.environ["GROQ_API_KEY"]

model = ChatGroq(model="llama3-8b-8192")

### Using Chains : Prompt Template and model and Output Parser with LCEL.

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser 

template1 = "Tell me {number} things about {topic}."

prompt1 = ChatPromptTemplate.from_template(template1)
prompt1.invoke({
    "number": 3,
    "topic": "dogs"
})

ChatPromptValue(messages=[HumanMessage(content='Tell me 3 things about dogs.')])

In [4]:
# This is for the template from messages

template2 = [
    ("system", "You are a advisor who will tell {number} things about:"),
    ("human", "Tell the topic in {topic}")
]
prompt2 = ChatPromptTemplate.from_messages(template2)
prompt2.invoke(
    {
        "number": 3,
        "topic": "dogs"
    }
)

ChatPromptValue(messages=[SystemMessage(content='You are a advisor who will tell 3 things about:'), HumanMessage(content='Tell the topic in dogs')])

In [5]:
chain = prompt1 | model | StrOutputParser()

# Running chain prompt1 -> model -> output parser will get the model response content

In [16]:
result = chain.invoke({
    "number": 3,
    "topic": "dogs"
})
print(result)

Here are three things about dogs:

1. **Dogs have a powerful sense of smell**: Dogs have up to 300 million olfactory receptors in their noses, compared to only 6 million in humans. This means they can detect smells that are too subtle for us to detect, and they use their sense of smell to navigate their environment and detect pheromones.
2. **Dogs are social animals**: Dogs are pack animals that thrive on social interaction with their human family and other dogs. They are naturally inclined to form close bonds with their pack members and enjoy activities like playing, cuddling, and going on walks together.
3. **Dogs are highly trainable**: Dogs are capable of learning a wide range of behaviors and tricks due to their ability to associate actions with rewards. They are often used as service animals, search and rescue dogs, and therapy dogs because of their ability to learn complex tasks and respond to commands.


### Structured output parser

In [10]:
# Using structured output parser

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question nicely.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [11]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question nicely.\n}\n```'

In [12]:
prompt3 = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)
prompt3

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question nicely.\n}\n```'}, template='answer the users question as best as possible.\n{format_instructions}\n{question}')

In [14]:
chain_structured_output = prompt3 | model | output_parser
chain_structured_output

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // answer to the user\'s question nicely.\n}\n```'}, template='answer the users question as best as possible.\n{format_instructions}\n{question}')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002BEFBCAD970>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002BEFBCAE390>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))
| StructuredOutputParser(response_schemas=[ResponseSchema(name='answer', description="answer to the user's question nicely.", type='string')])

In [15]:
chain_structured_output.invoke(
    {
        "what is Computer Vision?"
    }
)

{'answer': 'Computer Vision is a field of study that focuses on enabling computers to interpret and understand visual information from the world. It is a subfield of Artificial Intelligence (AI) that involves developing algorithms and techniques to extract meaningful information from images, videos, and other visual data. Computer Vision is used in a wide range of applications, including facial recognition, object detection, image classification, segmentation, and image generation. It has many practical uses in areas such as robotics, healthcare, security, and driver assistance systems.'}

The output is due to 
`
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question nicely.")
]
`

we can add more ResponseSchema in a list to create a more structured output as we want

### Chains: Under the Hood

In [29]:
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_core.prompts import PromptTemplate

template = "Tell me {number} things about {topic}."
prompt_template = PromptTemplate(
    template=template,
    input_variables=["topic"]
)

# Runnable lambda
prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

# Runnable sequence
chain = RunnableSequence(first=prompt, middle=[invoke_model], last=parse_output)

response = chain.invoke({
    "number": 3,
    "topic": "dogs"
})
print(response)

Here are three things about dogs:

1. **Dogs have a powerful sense of smell**: Dogs have up to 300 million olfactory receptors in their noses, compared to only 6 million in humans. This allows them to detect scents that are too faint for us to detect, and to track smells over long distances.
2. **Dogs are highly social animals**: Dogs are pack animals that thrive on social interaction. They are naturally inclined to live in groups and to form close bonds with their human family members. This is why they are often described as "man's best friend".
3. **Dogs can hear sounds that are too high for humans to hear**: Dogs can hear sounds with frequencies as high as 40,000 to 50,000 Hz, while humans can only hear sounds up to around 20,000 Hz. This means that dogs can pick up on high-pitched sounds like squeaky toys or even ultrasonic dog whistles that are beyond our range of hearing.


### Breakdown of what happened in each Runnable Lambda

#### prompt:
Here the input from chain.invoke is passed as `x` as a parameter in the lambda function and then the `prompt_template` `format_prompt` replaces the variables in the template to a string.

In [33]:
# This is what happen in the first RunnableLambda above

prompt = prompt_template.format_prompt(**{"topic": "dogs", "number": 3})
prompt

StringPromptValue(text='Tell me 3 things about dogs.')

#### invoke_model:
Here the output from first Runnable is pass to the invoke_model Runnable as a parameter x  in which the prompt is formatted to `langchains HumanMessage Format`

In [35]:
# This is what happen in the second RunnableLambda above

prompt.to_messages()

[HumanMessage(content='Tell me 3 things about dogs.')]

In [41]:
response = model.invoke(prompt.to_messages())
response

AIMessage(content='Here are three things about dogs:\n\n1. **Dogs are incredibly social animals**: Dogs are pack animals that thrive on interaction with their human family and other dogs. They are naturally inclined to form close bonds with their pack, which is why they often become so attached to their owners.\n2. **Dogs have a powerful sense of smell**: Dogs have up to 300 million olfactory receptors in their noses, compared to only 6 million in humans. This means they can detect smells that are too faint for us to detect, and they use their sense of smell to navigate their environment, detect food, and even detect health issues in their owners.\n3. **Dogs are highly trainable**: Dogs are able to learn a wide range of behaviors and tasks through training and conditioning. They are capable of learning hundreds of commands and tasks, from simple obedience behaviors like "sit" and "stay" to complex tasks like search and rescue, agility, and service work. Their ability to learn and adapt

In [39]:
from langchain_core.messages import HumanMessage

model.invoke([
    HumanMessage("Tell me 3 things about dogs.")
])

AIMessage(content='Here are three things about dogs:\n\n1. Dogs are highly social animals and thrive on interaction with their human family members. They are pack animals and have an instinct to belong to a group, which is why they often form strong bonds with their owners.\n2. Dogs have a unique sense of smell that is up to 10,000 times more sensitive than that of humans. They use their sense of smell to navigate their environment, detect food and toys, and even recognize their owners.\n3. Dogs are able to dream just like humans do. Studies have shown that dogs go through different stages of sleep, including REM sleep, during which they experience vivid dreams. This is thought to be an important part of their learning and memory consolidation process.', response_metadata={'token_usage': {'completion_tokens': 149, 'prompt_tokens': 18, 'total_tokens': 167, 'completion_time': 0.121219083, 'prompt_time': 0.003484648, 'queue_time': None, 'total_time': 0.12470373100000001}, 'model_name': 'l

#### output_parser:
Here the output from the model is passed as a parameter x and it is parsed i.e the content portion is extracted.

In [42]:
final_runnable_response = response.content
print(final_runnable_response)

Here are three things about dogs:

1. **Dogs are incredibly social animals**: Dogs are pack animals that thrive on interaction with their human family and other dogs. They are naturally inclined to form close bonds with their pack, which is why they often become so attached to their owners.
2. **Dogs have a powerful sense of smell**: Dogs have up to 300 million olfactory receptors in their noses, compared to only 6 million in humans. This means they can detect smells that are too faint for us to detect, and they use their sense of smell to navigate their environment, detect food, and even detect health issues in their owners.
3. **Dogs are highly trainable**: Dogs are able to learn a wide range of behaviors and tasks through training and conditioning. They are capable of learning hundreds of commands and tasks, from simple obedience behaviors like "sit" and "stay" to complex tasks like search and rescue, agility, and service work. Their ability to learn and adapt makes them valuable co

### Chain Extend
continually extend the chain

In [59]:
# simple create the half chain with LCEL and add more chains with Runnable Lambda

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

prompt_template = PromptTemplate(
    template="Tell me about {topic}. in exactly {number} words.",
    input_variables=["topic", "number"]
)

# prompt_template = ChatPromptTemplate.from_messages(
#     [
#         ("system", "Tell me about in {number} words exactly.."),
#         ("human", "Describe about {topic}")
#     ]
# )

parser = RunnableLambda(lambda x: x.content)
uppercase_output = RunnableLambda(lambda x: x.upper())
count = RunnableLambda(lambda x: f"The total number of words is: {len(x.split())}\n{x}")

chain = prompt_template | model | parser | uppercase_output | count

response = chain.invoke({
    "topic": "Natural Language Processing",
    "number": 100,
})

print(response)

The total number of words is: 103
NATURAL LANGUAGE PROCESSING (NLP) IS A SUBFIELD OF ARTIFICIAL INTELLIGENCE THAT ENABLES COMPUTERS TO UNDERSTAND, INTERPRET, AND GENERATE HUMAN LANGUAGE. IT INVOLVES DEVELOPING ALGORITHMS AND STATISTICAL MODELS THAT ANALYZE AND PROCESS HUMAN LANGUAGE DATA, SUCH AS TEXT OR SPEECH. NLP IS USED IN APPLICATIONS LIKE LANGUAGE TRANSLATION, SENTIMENT ANALYSIS, SPEECH RECOGNITION, AND TEXT SUMMARIZATION. IT INVOLVES TASKS LIKE TOKENIZATION, STEMMING, LEMMATIZATION, AND PARSING TO BREAK DOWN LANGUAGE INTO MEANINGFUL COMPONENTS, AND MACHINE LEARNING TO TRAIN MODELS ON LARGE DATASETS. NLP HAS NUMEROUS APPLICATIONS IN INDUSTRIES LIKE CUSTOMER SERVICE, HEALTHCARE, AND EDUCATION, AND HAS THE POTENTIAL TO REVOLUTIONIZE THE WAY HUMANS INTERACT WITH TECHNOLOGY.


In [61]:
# can use any output parser but the input and outout should be keep in mind while creating the chain
chain = prompt_template | model | StrOutputParser()
response = chain.invoke({
    "topic": "Natural Language Processing",
    "number": 100,
})
response

'Natural Language Processing (NLP) is a subfield of artificial intelligence that focuses on the interaction between computers and human language. NLP enables computers to understand, interpret, and generate human language, allowing for applications such as language translation, sentiment analysis, and text summarization. NLP involves various tasks, including tokenization, part-of-speech tagging, named entity recognition, and parsing. Techniques used in NLP include machine learning, deep learning, and rule-based systems. NLP has numerous applications, including chatbots, virtual assistants, and natural language interfaces for search engines and other systems. Its potential to improve human-computer interaction is vast and rapidly growing.'

### Chain Parallel

In [ ]:
#                         -> analyzechain1
# prompt -> model -> Parser               -> final response
#                         -> analyzechain2

In [109]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main 2 features of the product {product_name} with each feature not exceeding 10 words.")
    ]
)

def analyze_pros(features):
    """This function will return the prompt for the pros of the features.
    
    args: 
    features: str
    """
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            ("human", "Given these features:\n{features}\n, List the one main pros of these features.")
        ]
    )
    return pros_template.format_prompt(features=features)

def analyze_cons(features):
    """This function will return the prompt for the cons of the features.
    
    args:
    features: str
    """
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            ("human", "Given these features:\n{features}\n, List the one cons of these features.")
        ]
    )
    return cons_template.format_prompt(features=features)

In [110]:
# Testing the prompt

pros_prompt = analyze_pros("1. Good battery life\n2. Good camera quality\n3. Good display")
cons_prompt = analyze_cons("1. Expensive\n2. Heavy\n3. Not user-friendly")

print(pros_prompt)
print(cons_prompt)

messages=[SystemMessage(content='You are an expert product reviewer.'), HumanMessage(content='Given these features:\n1. Good battery life\n2. Good camera quality\n3. Good display\n, List the one main pros of these features.')]
messages=[SystemMessage(content='You are an expert product reviewer.'), HumanMessage(content='Given these features:\n1. Expensive\n2. Heavy\n3. Not user-friendly\n, List the one cons of these features.')]


In [111]:
# Creating branch for the Pros and Cons chain

# careful when chaining here, the output of the analyze_pros and analyze_cons should be the input for the model
pros_branch_chain = (
    RunnableLambda(lambda x:  analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

In [112]:
# Creating a combined chain using LCEL with RunnableParallel
from langchain_core.runnables import RunnableParallel

def combine_pros_cons(output_from_parallel_branches):
    print("Here is the output from the branch that was parallel:\n", output_from_parallel_branches)
    pros = output_from_parallel_branches["branches"]["pros"]
    cons = output_from_parallel_branches["branches"]["cons"]
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(
        branches={
            "pros": pros_branch_chain,
            "cons": cons_branch_chain
        }
    )
    # Here we get the dict from the parallel branch chain and this is how we get the pros and cons separately
    | RunnableLambda(lambda x: combine_pros_cons(x))
)

response = chain.invoke(
    {
        "product_name": "iPhone 13"
    }
)

Here is the output from the branch that was parallel:
 {'branches': {'pros': 'Based on the features, here are the one main pros for each:\n\n1. Improved Cameras with Enhanced Night Mode Capability:\n\t* Main Pro: Better low-light photography, allowing users to take high-quality photos even in dimly lit environments.\n2. Fastest Chip Ever - A15 Bionic Performance Boost:\n\t* Main Pro: Enhanced overall device performance, enabling smoother and faster execution of tasks, apps, and games.', 'cons': "Based on the features mentioned, here's one potential con for each:\n\n1. Improved Cameras with Enhanced Night Mode Capability:\nCon: The improved camera features may lead to increased power consumption, which could result in shorter battery life, especially when using the camera in low-light conditions.\n\n2. Fastest Chip Ever - A15 Bionic Performance Boost:\nCon: The increased processing power may also lead to faster battery drain, as the phone will be working harder to handle demanding tasks

In [113]:
print(response)

Pros:
Based on the features, here are the one main pros for each:

1. Improved Cameras with Enhanced Night Mode Capability:
	* Main Pro: Better low-light photography, allowing users to take high-quality photos even in dimly lit environments.
2. Fastest Chip Ever - A15 Bionic Performance Boost:
	* Main Pro: Enhanced overall device performance, enabling smoother and faster execution of tasks, apps, and games.

Cons:
Based on the features mentioned, here's one potential con for each:

1. Improved Cameras with Enhanced Night Mode Capability:
Con: The improved camera features may lead to increased power consumption, which could result in shorter battery life, especially when using the camera in low-light conditions.

2. Fastest Chip Ever - A15 Bionic Performance Boost:
Con: The increased processing power may also lead to faster battery drain, as the phone will be working harder to handle demanding tasks and games. This could result in users needing to charge their phone more frequently.


### Chain Branching:

In [114]:

#                                   -> Negative chain
#                                   -> positive chain  
# prompt -> classification model -> parser 
#                                   -> Neutral chain
#                                   -> Escalate chain

# here we will use RunnableBranch like checking if else condition for the chain.

In [144]:
from langchain_core.prompts import ChatPromptTemplate

positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service representative Alisa."),
        ("human", "The customer has given a positive feedback: {feedback} Please respond politely with thank you note,")
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service representative Jenny."),
        ("human", "The customer has given a negative feedback: {feedback} Please respond politely with an apology note.")
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service representative Erika."),
        ("human", "The customer has given a neutral feedback: {feedback} Please respond politely with a neutral note.")
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service representative Samantha."),
        ("human", "The customer has given a feedback: {feedback} Please escalate this to the higher authority.")
    ]
)

classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service representative."),
        ("human", "Classify the feedback: {feedback} as Positive, Negative, Neutral, Escalate, nofeedback.")
    ]
)

In [145]:
# creating branches
from langchain_core.runnables import RunnableBranch

branches = RunnableBranch(
    (
        # positive feedback chain
        lambda x: "positive" in x, # condition
        positive_feedback_template | model | StrOutputParser(),
    ),
    (
        # negative feedback chain
        lambda x: "negative" in x, # condition
        negative_feedback_template | model | StrOutputParser(),
    ),
    (
        # neutral feedback chain
        lambda x: "neutral" in x, # condition
        neutral_feedback_template | model | StrOutputParser(),
    ),
    # RunnableBranch must have default chain so here it is
    # otherwise error: RunnableBranch default must be runnable, callable or mapping.
    escalate_feedback_template | model | StrOutputParser()
)

In [146]:
classification_chain = classification_template | model | StrOutputParser()

# combining the classification and response generation into one chain
chain = classification_chain | branches

response = chain.invoke(
    {
        "feedback": "This product has good features but it is expensive."
    }
)

print(response)

Thank you so much for taking the time to share your thoughts about our product! We appreciate your honesty and value your feedback. It's great to hear that you've enjoyed some of the features and found them to be beneficial. However, we apologize that the high price was a drawback for you. We understand that our product may not fit every budget, and we're always looking for ways to improve and make it more accessible to our customers. Your input will definitely help us in our future product development and pricing strategy. Once again, thank you for your feedback, and we hope you'll continue to consider our product in the future.
